# Numerical exercise QM

Numerical solution of TDSE etc.

In [ ]:
'''Import necessary packages'''
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation
from IPython.display import HTML
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
'''Set common figure parameters:'''
newparams = {'axes.labelsize': 11, 'axes.linewidth': 1, 'savefig.dpi': 300, 
             'lines.linewidth': 1.0, 'figure.figsize': (10, 8),
             'ytick.labelsize': 10, 'xtick.labelsize': 10,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,}
plt.rcParams.update(newparams)

In [ ]:
'''Parameters and constants'''
h_bar = 1
m = 1
k_0 = 20
L = 20
N = 1000
omega = h_bar * k_0**2 / (2*m) # Obs. is this correct?? Jø nø se pa, håper det.
E = h_bar * omega # Is this correct?? Håper det også :^)
dx = L/(N-1)

In [ ]:
x = np.linspace(0, L, N) # Spacing should be almost dx

In [ ]:
def timestep(psi_im, psi_re, v, dt):
    """Performs one timestep of size dt
    Input:
        psi_im : Psi_im(x, t)
        psi_re : Psi_re(x, t + dt/2)
    Returns:
        psi_im : Psi_im(x, t + dt)
        psi_re : Psi_re(x, t + dt)
    """
    
    assert psi_im[0] == psi_im[-1] == psi_re[0] == psi_re[-1] == 0, "Psi should be zero for x = 0 and x = L"
    # Find psi_im(t+dt) given psi_im(t) and psi_re(t + dt/2)
    psi_im[1:-1] -= dt*(v[1:-1]*psi_re[1:-1]/h_bar - (psi_re[2:] - 2*psi_re[1:-1] + psi_re[:-2])*h_bar/(2*m*dx**2))
    
    # Find psi_re(t+ 3*dt/2) given psi_re(t+dt/2) and psi_im(t+dt) (remember, psi_im is now at t = t+dt)
    psi_re[1:-1] += dt*(v[1:-1]*psi_im[1:-1]/h_bar - (psi_im[2:] - 2*psi_im[1:-1] + psi_im[:-2])*h_bar/(2*m*dx**2))
    
    return psi_im, psi_re

In [ ]:
# Yo, Knært, jeg orket ikke tenke på hvor man burde plassere ting, så la den her. Kanskje dust, men den må hvertfall være før generate_psi()
''' Problem 1'''
x_s = 5 # Start position
x_f = L/2 + x_s # final position, later we will let the wave propagate til it reaches here, ellerno
sigma_x = 1.5

# Normalization
C = 1/np.sqrt(
    np.sum(np.exp(-(x-x_s)**2/(sigma_x**2)))*dx
)

v = np.zeros(N) # Zero potential
dt = 0.1 * h_bar / (h_bar**2 /(2*m*dx**2) + np.max(v)) # Timestep, much smaller than ..., see (12)

'''Parameters for propagation'''
v_g = h_bar*k_0/m # group velocity
sim_time = L/(2*v_g) # simulation time
num_iter = int(sim_time/dt) # number of iterations


In [ ]:
def generate_psi(x_s=x_s, sigma_x=sigma_x, num_iter=num_iter, N=N):
    """Generates inital values for psi
    Returns:
        psi_re
        psi_im"""
    # Arrays for storing psis at different points in time, need this later
    psi_re = np.zeros([num_iter, N])
    psi_im = np.zeros([num_iter, N])

    # see (8)
    psi_re[0, 1:-1] = C*np.exp(-(x[1:-1]-x_s)**2/(2*sigma_x**2)) * np.cos(k_0*x[1:-1] - omega*dt/2)
    psi_im[0, 1:-1] = C*np.exp(-(x[1:-1]-x_s)**2/(2*sigma_x**2)) * np.sin(k_0*x[1:-1])
    return psi_re, psi_im

In [ ]:
def generate_barrier(barrier_width, barrier_strength):
    return barrier_strength * np.heaviside(x*np.heaviside(-x +L/2 +barrier_width/2, 1) -L/2+barrier_width/2, 0)

In [ ]:
def simulate(x_s, sigma_x, barrier_width, barrier_strength, num_iter):
    '''
    Runs simulation for num_iter timesteps, using the timestep function.
    
    Input:
        v: potential vector
        x_s: starting position
        sigma_x: standard deviation for the Gaussian
        num_iter: number of interations (timesteps)
    Output:
        psi_real: vector with the real parts of the WF at each timestep
        psi_imag: vector with the imaginary parts of the WF at each timestep
    '''
    C = 1/np.sqrt(
    np.sum(np.exp(-(x-x_s)**2/(sigma_x**2)))*dx
    )
    
    v = generate_barrier(barrier_width, barrier_strength)
    psi_re, psi_im = generate_psi(x_s=x_s, sigma_x=sigma_x, num_iter=num_iter)

    for i in range(num_iter-1):
        psi_im[i+1], psi_re[i+1] = timestep(psi_im[i], psi_re[i], v, dt) #todo: pass more sensible dt
        
    return psi_re, psi_im

In [ ]:
def simulate_and_plot(x_s, sigma_x, barrier_width, barrier_strength, num_iter):
    """Simulates and plots inital and final state for psi_re"""
    psi_re, _ = simulate(x_s, sigma_x, barrier_width, barrier_strength, num_iter)
    plt.plot(x, psi_re[0], label="Initial Re")
    plt.plot(x, psi_re[-1], label="Final Re")
    plt.legend()

In [ ]:
# Helper function for animation

def _get_ims_3D(psi_1, psi_2, ax):
    ims = []
    for i in range(0, len(psi_1), 100):
        ln1 = plt.plot(x, psi_1[i], psi_2[i], 'tab:blue')
        ims.append(ln1)
    return ims

def _get_ims_2D(psi_1, psi_2, ax):
    ims = []
    for i in range(0, len(psi_1), 100):
        ln1, = ax.plot(x, psi_1[i], 'tab:orange')
        ln2, = ax.plot(x, psi_2[i], 'tab:blue')
        ims.append((ln1, ln2,))
    return ims

In [ ]:
def animate(psi_1, psi_2, in_3D=False):
    """Animates psi_1 and psi_2
    Parameters:
        psi_1, psi_2 : ndarray, shape (num_iterations, N)
        in_3D        : bool, animate in 3D
    Returns:
        HTML5 video"""
    assert len(psi_1)==len(psi_2), "psi_1 and psi_2 must have the same length!"
    fig = plt.figure(figsize=(8, 6))
    if in_3D:
        ax = fig.add_subplot('111', projection='3d')
        ims = _get_ims_3D(psi_1, psi_2, ax)
    else:
        ax = fig.add_subplot('111')
        ims = _get_ims_2D(psi_1, psi_2, ax)
        
    anim = animation.ArtistAnimation(fig, ims, interval=60)
    plt.close(anim._fig)
    return HTML(anim.to_html5_video())

def simulate_and_animate(x_s, sigma_x, barrier_width, barrier_strength, num_iter, in_3D=False):
    psi_re, psi_im = simulate(x_s, sigma_x, barrier_width, barrier_strength, num_iter)
    
    return animate(psi_re, psi_im, in_3D=in_3D)

In [ ]:
simulate_and_animate(5, 2, L/5, 9*E/10, num_iter)

## Problem 1

We start by calculating the initial wave function, normalising it, and plotting the real and imaginary parts. We also plot the initial probability distribution for the particles position given by $\mid{\Psi(x,t)}\mid^2$.

In [ ]:
psi_re, psi_im = generate_psi()

In [ ]:
''' Plot start state, real and imaginary'''
plt.plot(x, psi_re[0], label="Initial Re")
plt.plot(x, psi_im[0], label="Initial Im")
plt.title('Initial wave function')
plt.legend()
simulated_time = dt*num_iter
print(f"Simulated time: {simulated_time}s")

In [ ]:
'''Initial probability density'''
initial_prob_density = np.square(np.absolute(psi_re[0] + psi_im[0]*1j))
plt.plot(x, initial_prob_density, label='')
plt.title('Probability density in the initial state')
plt.show()

## Problem 2

We now let the wave packet propagate from its starting position $x_s = 5$ to $x_f = 15$.

In [ ]:
simulate_and_plot(x_s, sigma_x, 0, 0, num_iter)

In [ ]:
'''Simulation for more sigma_x's'''
sigma_xs = [0.5, 1.0, 2.0]

for i, sigma_x in enumerate(sigma_xs):
    plt.subplot(f"31{i+1}")
    simulate_and_plot(x_s, sigma_x, 0, 0, num_iter)

## Introducing a barrier

In [ ]:
'''Defines a barrier'''
l = L/20 # barrier width
v_0 =  3*E/4 # barrier height
psi_re, psi_im = simulate(x_s, sigma_x, l, v_0, num_iter)

In [ ]:
'''Plot start and final state'''
plt.plot(x, psi_re[0], label="Initial")
plt.plot(x, psi_re[-1], label="FInal")
plt.legend()

In [ ]:
'''Animation'''
animate(psi_re, psi_im)

In [ ]:
'''Calculates P(right) and P(left), probability of being right and left of the barrier'''
mid_i = int(N/2) # Index of center

P_right = np.sum((psi_re**2+psi_im**2)[[0, -1], mid_i:], axis=1)*dx
P_left = np.sum((psi_re**2+psi_im**2)[[0, -1], :mid_i], axis=1)*dx

print(f"P_right: {P_right}\nP_left: {P_left}\nSum: {P_right + P_left}")

In [ ]:
print(f"Probability of transmission: {P_right[1]}")

In [ ]:
'''Calculates transmission probability as a function of barrier height'''
l = L/40
fraction = np.linspace(0, 3/2, 50)
P_trans = []

for f in fraction:
    psi_re, psi_im = simulate(x_s, sigma_x, l, f*E, num_iter)
    P_trans.append(np.sum((psi_re**2+psi_im**2)[-1, mid_i:])*dx)

In [ ]:
plt.plot(fraction, P_trans)

In [ ]:
# Find transmission probability as a function of barrier width
v_0 =  9*E/10 # barrier height

widths = np.geomspace(0.01, L/10, 50)
P_trans = []

for l in widths:
    psi_re, psi_im = simulate(x_s, sigma_x, l, v_0, num_iter)
    P_trans.append(np.sum((psi_re**2+psi_im**2)[-1, mid_i:])*dx)
plt.plot(widths/L, P_trans, 'o-') # Note that widths are plotted as fractions of L
plt.show()